# Parabolic model problem (Unit 3.1)

We are solving the unsteady heat equation

$$\text{find } u:[0,T]\rightarrow H_{0,D}^1 \hspace{1em} \int_\Omega \partial_t u v + \int_\Omega \epsilon \nabla u \cdot \nabla v + b \cdot \nabla u v = \int f v $$ 

$$\forall v \in H_{0,D}^1 \hspace{2em} u(t=0) = u_0 $$

with a suitable advective field b
(the wind).

I think this is a weak form of the equation:

$$u_t - \epsilon \Delta u + b \nabla u = f \text{ on } \Omega$$
$$u = 0 \text{ on } \partial \Omega$$

In [1]:
import netgen.gui
%gui tk
import tkinter
from math import pi
from ngsolve import *
from netgen.geom2d import SplineGeometry


We generate geometry $(−1,1)^2$, mesh (Dirichlet boundaries everywhere):

In [12]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")
u,v = fes.TnT()

time = 0.0
dt = 0.001
eps = 0.01

We define the field $b$ (the wind) as $b(x,y)=(2y(1−x2),−2x(1−y2))$.

In [22]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind")


We define and assemble bilinear forms for convection-diffusion stiffness and mass matrix separately.

Note that we choose a non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern (a is already non-symmetric).

In [14]:
a = BilinearForm(fes, symmetric=False)
a += SymbolicBFI (eps*grad(u)*grad(v) + b*grad(u)*v)
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += SymbolicBFI (u*v)
m.Assemble()


## Implicit Euler method

We want to use a simple implicit Euler time stepping method, i.e.
$$\underbrace{(M+\Delta t\ A)}_{M^{*}}u^{n+1}=M u^n+f^{n+1}\Delta t$$

or in an incremental form:
$$M^{*}(u^{n+1}−u^n)=−\Delta t\ A u^n+f^{n+1}\Delta t.$$

The incremental form has the advantage that $u^{n+1}−u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).

For the time stepping method we want to set up linear combinations of matrices.

(Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of the same size and sparsity pattern as m.mat:

### Notes:
I modified the above to show that $f^{n+1}$ should be multiplied by $\Delta t$  This is the only way the analysis makes sense, and it's coded this way too.

If $\epsilon$ is set to zero, $M^*$ is still invertible, but the solution looks fractured not smooth

In [15]:
mstar = m.mat.CreateMatrix()
# print(mstar)

To access the entries we use the vector of nonzero-entries:

In [16]:
# print(mstar.AsVector())


In [17]:
print(mstar.nze)
print(len(mstar.AsVector()))

11257
11257


Using the vector we can build the linear combination of the a and m matrices

In [18]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

We set the r.h.s. $f=exp(−6((x+12)2+y2))−exp(−6((x−12)2+y2))$

In [19]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f")
f += SymbolicLFI(gaussp*v)
f.Assemble()

and the initial data: $u_0=(1−y^2)x$

In [20]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,"u")

In [21]:
tstep = 1 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 1.000000000000000776


Alternative version with iterative solvers

- For a factorization of $M^∗\ (M^{∗−1})$ we require a sparse matrix $M^*$
- To store $M^∗$ as a sparse matrix requires new storage (and same memory layout of A and M)
- For iterative solvers we only require the matrix (and preconditioner) applications
- ```mstar = m.mat + dt * a.mat``` has no storage but matrix-vector multiplications

iterative solver version (with Jacobi preconditining):


In [17]:
mstar_alt = m.mat + dt * a.mat
premstar_alt = m.mat.CreateSmoother() + dt * a.mat.CreateSmoother()
invmstar_alt = CGSolver(mstar_alt,premstar_alt,printrates=True)

print(premstar_alt)

Sum of
Scale a = 1
Print base-matrix
Scale b = 1
Scale with 0.001:
Print base-matrix



## Supplementary 1: time-dependent r.h.s. data

Next, we want to consider time-dependent r.h.s. data $f=f(t)$:

To this end, we introduce a ```Parameter``` t representing the time.

A Parameter is a constant CoefficientFunction the value of which can be changed with the ```Set``` function.

In [21]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [22]:
omega=1
gausspt = exp(-6*((x+sin(omega*t))*(x+sin(omega*t))+y*y))-exp(-6*((x-sin(omega*t))*(x-sin(omega*t))+y*y))
Draw(gausspt,mesh,"ft")
time = 0.0
while time < 10 - 0.5 * dt:
    t.Set(time)
    tt = time
    Redraw(blocking=True)
    time += 1e-4

Accordingly we define a different linear form which then has to be assembled in every time step.

In [23]:
ft = LinearForm(fes)
ft += SymbolicLFI(gausspt*v)
time = 0.0
t.Set(0.0)
gfu.Set((1-y*y)*x)
#gfu.Set(CoefficientFunction(0))
Draw(gfu,mesh,"u")

In [24]:
tstep = 20 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    ft.Assemble()
    res.data = dt * ft.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 20.00000000000146876


## Supplementary 2: Time dependent boundary conditions

Now, we want to consider changing boundary conditions $u|_{\partial \Omega} = u_D(t)$ and set $f=0$.

We use the simple implicit Euler time stepping method again for this problem. Now, we directly use the non-incremental form:
$$M^* u^{n+1}=(M+\Delta t\ A)u^{n+1}=M u^n$$
We homogenize w.r.t. to the boundary conditions, i.e. we split $$u^{n+1}=u_0^{n+1}+u_D^{n+1}$$ where $u_D^{n+1}$ is some function in the FESpace with the correct boundary condition and $u_0^{n+1}$ is the sought for function with zero boundary values:
$$M^* u_0^{n+1} = Mu^n - M^* u_D^{n+1}$$

In [25]:
uD = CoefficientFunction( [(1-x*x)*IfPos(sin(0.3*pi*t),sin(0.3*pi*t),0),0,0,0])
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0


In [26]:
tstep = 2 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    #non-incremental form 
    res.data = m.mat * gfu.vec
    gfu.Set(uD,BND)
    res.data -= mstar * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 1.999999999999890576


## Supplementary 3: Singly diagonally implicit Runge-Kutta methods

We consider more sophisticated time integration methods, SDIRK methods. To simplify presentation we set $f=0$.

SDIRK methods for the semi-discrete problem $\cfrac{d}{dt}u=M^{−1}F(u)=−M^{−1}\cdot A u$ are of the form:
$u^{n+1}=u_n+\Delta t\ M^{−1} \sum_{i=0}^{s−1} b_i k_i$ with $k_i=−A u_i$ where $u_i$ is the solution to 
$$(M+a^∗\Delta t A) u_i=M u^n−\Delta t \sum_{j=0}^{i−1} a_{ij}k_j, \hspace{2em} i=0,\dots,s−1.$$
The coefficients $a$,$b$ and $c$ are stored in the butcher tableau: 
$$
\begin{array}{c|cccc}
c_0 & a_{00} & 0 & \ddots& \\
c_1 & a_{10} & a_{11} & 0 & \ddots \\
\vdots & \vdots & \vdots & \ddots & 0\\
c_{s-1} & a_{s-1,0} & a_{s-1,1} & \dots& a_{s-1,s-1} \\ \hline
 & b_{0} & b_1 & \dots& b_{s-1} \\
\end{array}
$$

For an SDIRK method we have $a^\ast =a_{ii}, ~i=0,\dots,s−1$.


Simplest example: Implicit Euler
$$
\begin{array}{c|c}
1 & 1 \\ \hline
 & 1  \\
\end{array}
$$

In [27]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1

We can use for example the 2 stage SDIRK (order 3) method
$$
\begin{array}{c|cc}
p & p & 0 \\
1-p & 1-2p & p \\ \hline
 & 1/2 & 1/2 \\
\end{array}
$$
with $p = \frac{3 - \sqrt{3}}{6}$.

In [28]:
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p


We can use for example the 5 stage SDIRK (order 4) method
$$
\begin{array}{c|cccc}
1/4 & 1/4  \\
3/4 & 1/2 & 1/4 & \\
11/20 & 17/50 & -1/25 & 1/4 \\
1/2 & 371/1360 & -137/2720 & 15/544 & 1/4 \\
1 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\ \hline
 & 25/4 & -49/48 & 125/16 & -85/12 & 1/4 \\
\end{array}
$$

In [29]:
 class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4    

### SDIRK2:

In [30]:

butchertab = sdirk2()    
    
rhsi = gfu.vec.CreateVector()
Mu0 = gfu.vec.CreateVector()
ui = gfu.vec.CreateVector()
k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]

We have to update the $M^∗$ matrix and reset initial data

In [31]:
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.2
visoptions.deformation = 0
visoptions.autoscale = 0

mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
invmass = m.mat.Inverse(freedofs=fes.FreeDofs())

In [32]:
tstep = 2 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    Mu0.data = m.mat * gfu.vec
    for i in range(butchertab.stages):
        # add up the ks as prescribed in the butcher tableau
        rhsi.data = Mu0
        for j in range(0,i):
            rhsi.data += dt * butchertab.a[i][j] * k[j]
        
        # Solve for ui (with homogenization for the boundary data)
        t.Set(time+t_intermediate+butchertab.c[i]*dt)
        gfu.Set(uD,BND)
        ui.data = gfu.vec
        rhsi.data -= mstar * ui
        ui.data += invmstar * rhsi
        
        # compute k[i] from ui
        k[i].data = - a.mat * ui
        
    t_intermediate += dt
    t.Set(time+t_intermediate)
    
    # Adding up the ks:
    gfu.Set(uD,BND)
    Mu0.data -= m.mat * gfu.vec
    for i in range(0,butchertab.stages):
        Mu0.data += dt * butchertab.b[i] * k[i]
    gfu.vec.data += invmass * Mu0
        
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    
print("")
time+=t_intermediate    

 1.999999999999890576


## Supplementary 4: VTK Output ("exporting the nice pictures")

- see also https://ngsolve.org/blog/ngsolve/2-vtk-output
- or py_tutorials/vtkout.py (ngsolve repository)

Outputting the nice pictures to vtk (to visualize with paraview):


In [26]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],
                names=["sol"],
                filename="vtk_example1",
                subdivision=3)
vtk.Do()


In [27]:
%%bash
ls vtk_example1.vtk

vtk_example1.vtk


You can also export vector fields:

In [28]:
vtk = VTKOutput(ma=mesh,coefs=[gfu,grad(gfu)],names=["sol","gradsol"],filename="vtk_example2",subdivision=3)
vtk.Do()

In [29]:
#%%bash
#paraview vtk_example2.vtk

And time dependent data:

In [30]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],names=["sol"],filename="vtk_example3",subdivision=3)
vtk.Do()

gfu.Set((1-y*y)*x)

time = 0
tstep = 0.3 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
i = 0
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    i += 1
    if (i%10 == 0):
        vtk.Do()
print("")

 0.300000000000000276


In [31]:
#%%bash
#paraview vtk_example3_..vtk